### Note: This is only applicable to Rhino. The principles of parallel computing are general and can be used to accelerate code on any system, but the implementations here are designed to interact with the scheduler used on the Computational Memory Lab cluster.

## Parallel Computing with Dask

<img src="https://hpc.llnl.gov/sites/default/files/styles/with_sidebar_1_up/public/nodesNetwork.gif?itok=TBqDQmx0">

A node is a like a computer within a much bigger computer!

**CMLDask wrapper**:
* See https://github.com/pennmem/cmldask for wrapper package created for convenience using Rhino + typical parallel setup for our analyses
* Check out [Dask](https://docs.dask.org/en/stable/) and [Dask distributed](https://distributed.dask.org/en/stable/) for more documentation about the underlying implementation. In particular, we're using the [Futures API](https://docs.dask.org/en/stable/futures.html?highlight=futures) for managing parallel tasks and [dask-jobqueue](http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SGECluster.html#dask_jobqueue.SGECluster) to connect with the Sun Grid Engine (SGE) scheduler on rhino. 
* Note that this package is intended for convenience and will not accomodate very specific computational needs that demand complex parallel architectures. You will need to use dask directly and create your own `Client()` instances that are more closely tailored to your needs.

### Basic usage works as follows:

In [1]:
import cmldask.CMLDask as da

/usr/global/Anaconda/2019-10/envs/cml37/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
import os
import cmldask
from dask.distributed import wait, as_completed, progress

def squared(x):
    return x**2

logdir = os.path.join(os.path.abspath(os.curdir), 'dask_appendix_logs')
if not os.path.exists(logdir): os.mkdir(logdir)
client = da.new_dask_client("test_dask", "1GB", log_directory=logdir, max_n_jobs=5)

Unique port for rdehaan is 51474
{'dashboard_address': ':51474'}
To view the dashboard, run: 
`ssh -fN rdehaan@rhino2.psych.upenn.edu -L 7000:192.168.86.143:33610` in your local computer's terminal (NOT rhino) 
and then navigate to http://localhost:7000 in your browser


/usr/global/Anaconda/2019-10/envs/cml37/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 51474 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33610 instead
  f"Port {expected} is already in use.\n"


In [6]:
# where cluster run logs will be stored
logdir

'/home1/rdehaan/projects/DataMemoryBrains/dask_appendix_logs'

**Note that you need to explicitly shut down this client (`client.shutdown()`) or shut down your jupyter kernel before running this cell again, or you might leave workers "stranded" without access to them (since that access is provided by the Client you would overwrite)**

You have options when setting up your Dask cluster/client:

In [10]:
da.new_dask_client?

Signature:
da.new_dask_client(
    job_name,
    memory_per_job,
    max_n_jobs=100,
    threads_per_job=1,
    processes_per_job=1,
    adapt=True,
    queue='RAM.q',
    walltime='1500000',
    local_directory=None,
    log_directory=None,
    scheduler_options={'dashboard_address': ':51391'},
    **kwargs,
)
Docstring:
Returns a new dask client instance with associated dashboard for
monitoring jobs. The default method assumes a very basic use case - that
is, embarassingly parallel tasks - in which the user simply wants to speed
up independent computations by running them in parallel. There is just one
single-threaded process per job. The thread/process parameters are
configurable, but will lead to more complicated parallel executions
which might be hard to track. Workers might share memory instead of
being independent.
Leave the defaults unless you have some specific reason to believe another
configuration will help with your use case.

Parameters
----------
job_name : gives every j

## Parallel computing tips

* Jobs are still subject to memory limitations, so you may need to **break up large processes into smaller chunks.** For example, each job could correspond to analyzing one session, instead of one subject (this should be done, e.g., for all Morlet wavelet analyses). 
* It is often useful to save the output of each job in a dedicated directory, and sometimes useful to save intermediate values to aid in debugging or later nonparallel analyses. The Python "os" library can be helpful here. 
* **Be respectful!** There are only so many cores available to your classmates, the Kahana lab, and our collaborators across the country.
* **Limit typical jobs to 5 cores or less**. Heavy usage means fewer resources for other users, and due to shared disk resources might actually slow down all jobs overall. Please ask for permission before using more.
* You can always use the '**qdel**' command in Terminal, followed by your job number, to kill any of your old jobs that may be wasting rhino's resources. 
* Use the '**qstat**' command in Terminal to see cluster usage information for all users. The **'qstat | grep your_username'** command will just show the jobs associated with your username
* Each rhino2 node has ~128 GB of memory and ~40 cores.


### Simple Dask Futures Usage

In [4]:
import numpy as np
from time import sleep

# functions to run on the cluster
def add(a, b):
    sleep(np.random.randint(0, 10))
    return a + b

def error_add(a, b):
    sleep(np.random.randint(0, 10))
    if a % 2:
        raise ValueError
    return a + b

# launch the jobs
futures = client.map(add, range(40), range(40))
progress(futures)

VBox()

In [5]:
# just for illustration; the log files are just text files, so it's typically easier to open them up manually
with open(os.path.join(logdir, os.listdir(logdir)[0]), 'r') as f:
    log = f.read()
print(log)

distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.86.139:40800'
distributed.diskutils - INFO - Found stale lock file and directory '/home1/rdehaan/dask-worker-space/dask-worker-space/worker-jiq0qy7u', purging
distributed.worker - INFO -       Start worker at: tcp://192.168.86.139:35492
distributed.worker - INFO -          Listening to: tcp://192.168.86.139:35492
distributed.worker - INFO -          dashboard at:       192.168.86.139:38817
distributed.worker - INFO - Waiting to connect to: tcp://192.168.86.143:37742
distributed.worker - INFO - -------------------------------------------------
distributed.worker - INFO -               Threads:                          1
distributed.worker - INFO -                Memory:                   0.93 GiB
distributed.worker - INFO -       Local Directory: /home1/rdehaan/dask-worker-space/dask-worker-space/worker-vfsoljtz
distributed.worker - INFO - -------------------------------------------------
distributed.worker - INFO -      

In [5]:
client.map?

Signature:
client.map(
    func,
    *iterables,
    key=None,
    workers=None,
    retries=None,
    resources=None,
    priority=0,
    allow_other_workers=False,
    fifo_timeout='100 ms',
    actor=False,
    actors=False,
    pure=None,
    batch_size=None,
    **kwargs,
)
Docstring:
Map a function on a sequence of arguments

Arguments can be normal objects or Futures

Parameters
----------
func : callable
    Callable to be scheduled for execution. If ``func`` returns a coroutine, it
    will be run on the main event loop of a worker. Otherwise ``func`` will be
    run in a worker's task executor pool (see ``Worker.executors`` for more
    information.)
iterables : Iterables
    List-like objects to map over.  They should have the same length.
key : str, list
    Prefix for task names if string.  Explicit names if list.
pure : bool (defaults to True)
    Whether or not the function is pure.  Set ``pure=False`` for
    impure functions like ``np.random.random``.
    See :ref:`pur

### Handling Errors

Now with errors

In [12]:
futures = client.map(error_add, range(40), range(40))

Wait for all jobs to finish, check for errors

In [13]:
wait(futures)
errors = da.get_exceptions(futures, range(40))
errors

,exception,traceback_obj
param,,
1,ValueError(),<traceback object at 0x2aca5a032280>
3,ValueError(),<traceback object at 0x2aca5a0bc140>
5,ValueError(),<traceback object at 0x2aca5a13bcd0>
7,ValueError(),<traceback object at 0x2aca5a10b640>
9,ValueError(),<traceback object at 0x2aca5a146780>
11,ValueError(),<traceback object at 0x2aca5a152e60>
13,ValueError(),<traceback object at 0x2aca5a1889b0>
15,ValueError(),<traceback object at 0x2aca5a134320>
17,ValueError(),<traceback object at 0x2aca5a18cd70>


Pick out the index where you want to view the traceback message

In [14]:
da.print_traceback(errors, 5)

  File "<ipython-input-11-492535f77c2d>", line 11, in error_add
    raise ValueError


Notice gathering these doesn't work because there are errors

In [15]:
client.gather(futures)

ValueError: 

Instead, let's filter for successful ones

In [16]:
good_futures = da.filter_futures(futures)
client.gather(good_futures)

[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76]

## IMPORTANT: Shutdown your client (or restart your kernel, which will do so automatically)

In [ ]:
client.shutdown()

## Recommended Best Practices

1. Watch your memory usage. Jobs that go over memory limits will error out. Delete unnecessary variables or overwrite variables to reduce memory overhead.
2. Saving out **dimensionally reduced** intermediate outputs. 
    * Clusters can be unreliable and have outages, and jobs can get killed. Saving results from each job ensures that all results will not lost if a run dies partway through. 
    * You can write code to check if the results for a particular job have successfully returned and only recompute results for failed jobs.
3. Avoid heavy disk usage by reading from and writing to a smaller number of medium to large-sized files rather than a large number of small files. The network/disk bandwidth on Rhino is a major bottleneck. If you use file IO (Input/Output, i.e., reading and writing to files) heavily, it will not only makes others' work slow, it can slow down your own analyses substantially.


In [ ]:
# example code with the Pickle library, which allows for saving and loading (most) objects in Python
# the pickle library is not recommended for long-term storage since it depends on having the same object 
# code/package versions available when loading an object that were present when the object was saved
import pickle
import numpy as np

arr = np.arange(1000)

# save output
with open('arr_test.pkl', 'wb') as f:
    pickle.dump(arr, f)
    
# load output
with open('arr_test.pkl', 'rb') as f:
    arr_loaded = pickle.load(f)

# confirm that loaded == saved
np.all(arr == arr_loaded)

In [1]:
class Settings():
  '''settings = Settings()
     settings.somelist = [1, 2, 3]
     settings.importantstring = 'saveme'
     settings.Save()

     settings = Settings.Load()
  '''
  def __init__(self, **kwargs):
    for k,v in kwargs.items():
      self.__dict__[k] = v

  def Save(self, filename='settings.pkl'):
    import pickle
    with open(filename, 'wb') as fw:
      fw.write(pickle.dumps(self))

  def Load(filename='settings.pkl'):
    import pickle
    return pickle.load(open(filename, 'rb'))

  def __repr__(self):
    return ('Settings(' +
      ', '.join(str(k)+'='+repr(v) for k,v in self.__dict__.items()) +
      ')')

  def __str__(self):
    return '\n'.join(str(k)+': '+str(v) for k,v in self.__dict__.items())


In [2]:
settings = Settings()
settings.experiment = 'FR1'
settings.logbase = 'totalwords_'
settings.sub_list = ['R1341T', 'R1391T', 'R1395M', 'R1467M']

settings.Save('totalwords.pkl')
print(settings)

experiment: FR1
logbase: totalwords_
sub_list: ['R1341T', 'R1391T', 'R1395M', 'R1467M']


In [3]:
import cmldask.CMLDask as da
from dask.distributed import wait, as_completed, progress
from cmlreaders import CMLReader, get_data_index
import numpy as np
import pandas as pd
import datetime
import traceback

def TotalWords(sub):
  logfile = f'{settings.logbase}{sub}.txt'
  
  df = get_data_index('all')
  df = df[df['subject']==sub]
  df = df[df['experiment']==settings.experiment]

  total_words = 0
  for df_sess in df.itertuples():
    try:
      df_sess = df_sess._asdict()
      # Get parameters from the session DataFrame
      exp = df_sess['experiment']
      sess = df_sess['session']
      # Prepare the reader
      reader = CMLReader(sub, exp, sess, df_sess['montage'], df_sess['localization'])
      # Get word events
      evs = reader.load('task_events')
      word_evs = evs[evs['type']=='WORD']
      # Do our analysis!
      total_words += len(word_evs)
    except Exception as e:
      # Log the exception to a subject-labeled filename,
      # along with a label of subject, experiment, and session.
      with open(logfile, 'a') as fw:
        date = datetime.datetime.now().strftime('%F_%H-%M-%S')
        fw.write(f'{date}: {sub}, {exp}, {sess}\n' +
                 ''.join(traceback.format_exception(type(e), e, e.__traceback__)))

    
  # Save the result.
  np.save('wordcount_'+sub+'.npy', [total_words])
  return total_words

/usr/global/Anaconda/2019-10/envs/cml37/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [4]:
# Run in parallel
client = da.new_dask_client("test_dask", "4GB", max_n_jobs=5)
futures = client.map(TotalWords, settings.sub_list)

Unique port for ryan.colyer is 51391
{'dashboard_address': ':51391'}
To view the dashboard, run: 
`ssh -fN ryan.colyer@rhino2.psych.upenn.edu -L 7000:192.168.86.143:51391` in your local computer's terminal (NOT rhino) 
and then navigate to http://localhost:7000 in your browser


In [5]:
# Wait for results, check errors.
wait(futures)
#errors = da.get_exceptions(futures, range(40))
#errors

DoneAndNotDoneFutures(done={<Future: finished, type: int, key: TotalWords-926e2b75e568cfaf62c2df3f1b60f651>, <Future: finished, type: int, key: TotalWords-7bc1ebcfc280b90f3d50c762bf05a7f9>, <Future: finished, type: int, key: TotalWords-67821e81b45a688188187f1157db14d8>, <Future: finished, type: int, key: TotalWords-40a3f70f7b8f36a6ab1b4e3b44ad16ec>}, not_done=set())

In [6]:
good_futures = da.filter_futures(futures)
client.gather(good_futures)

[1260, 468, 936, 156]

In [7]:
settings = Settings.Load('totalwords.pkl')
for sub in settings.sub_list:
  total_words = np.load('wordcount_'+sub+'.npy')
  print(f'{sub} had {total_words} words')

R1341T had [1260] words
R1391T had [468] words
R1395M had [936] words
R1467M had [156] words


Happy parallel computing!

**Exercise: Write a parallel function that returns the number of (bipolar) electrodes for every subject in the RAM example dataset (ignore localization and montage details). Run with 5 jobs and 1 core per job. This will require you to integrate the data loading procedures you learned earlier with the dask framework exhibited using the toy examples above**

This example is purely for illustration and this task is likely not much faster with multiple nodes than with a single node.